Name: Bedirhan Caldir

I hereby declare that I observed the honour code of the university when preparing the homework.

## Pr?gr?mm?ng?H?m?w?rk

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [1]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [2]:
import csv
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
for c,p in zip(alphabet,T[letter2idx['a']]):
    print(c,p)

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

a 0.0002835
b 0.0228302
c 0.0369041
d 0.0426290
e 0.0012216
f 0.0075739
g 0.0171385
h 0.0014659
i 0.0372661
j 0.0002353
k 0.0110124
l 0.0778259
m 0.0260757
n 0.2145354
o 0.0005459
p 0.0195213
q 0.0001749
r 0.1104770
s 0.0934290
t 0.1317960
u 0.0098029
v 0.0306574
w 0.0088799
x 0.0009562
y 0.0233701
z 0.0018701
. 0.0715219


#### 1 - Sampling random strings

In [3]:
import numpy as np

def generate_random_string(N):
    string = '.' # initialized with '.' since it represents string beginning (will be omited at the end)
    for i in range(N):
        probs = np.array(list(map(float, T[letter2idx[string[-1]]])))
        string += alphabet[np.random.choice(len(alphabet), p=probs/np.sum(probs))]
    return string[1:]

for i in range(10):
    N = np.random.randint(10, 20)
    print('For N={} ->'.format(N), generate_random_string(N))

For N=11 -> of.wonconda
For N=14 -> se..alened.fin
For N=11 -> ath.pire.ck
For N=13 -> .shentheaimex
For N=19 -> s.cromo.whe.wa.cawh
For N=16 -> vict.tunt.t.al.a
For N=15 -> y.eve.e.sthend.
For N=15 -> s.mitaveed.bawn
For N=11 -> sps.ir..st.
For N=16 -> .oul.win.sest.ty


#### 2 - Missing letter inference

In [4]:
# Since HMM assumes that a state only depends on its previous state, 
# an unknown letter will be using its previous letter..

def infer_missing_letters(string, missing_indicator='_', nonletter_indicator='.'):
    result = '.'
    for letter in string:
        letter = letter.lower()
        if letter == missing_indicator:
            probs = np.array(list(map(float, T[letter2idx[result[-1]]])))
            result += alphabet[np.random.choice(len(alphabet), p=probs/np.sum(probs))]
        else:
            result += letter
    return result[1:]

print('Random letter inferences:')
for string in test_strings:
    inferred = infer_missing_letters(string)
    print(string, '->', inferred)

Random letter inferences:
th__br__n.f_x. -> thosbr.cn.f.x.
_u_st__n_.to_be._nsw_r__ -> tucsthind.towbe.mnswar.t
i__at_._a_h_n_._e_r_i_g -> icoati.havhant.se.riisg
q___t.___z._____t.__.___.__. -> qulyt.on.z.alfert.ro.hon.ce.


#### 3 - Missing letter inference with most likely letter

In [5]:
def infer_missing_letters_with_argmax(string, missing_indicator='_', nonletter_indicator='.'):
    result = '.'
    p = 0
    for letter in string:
        letter = letter.lower()
        if letter == missing_indicator:
            probs = np.array(list(map(float, T[letter2idx[result[-1]]])))
            result += alphabet[np.argmax(probs)]
        else:
            result += letter
        p += np.log(float(T[letter2idx[result[-1]]][letter2idx[result[-2]]]))
    return result[1:], p

print('The most likely letter inferences:')
for string in test_strings:
    inferred, p = infer_missing_letters_with_argmax(string)
    print(string, '->', inferred, 'with log-joint of', p)

The most likely letter inferences:
th__br__n.f_x. -> the.bre.n.f.x. with log-joint of -52.35832852820339
_u_st__n_.to_be._nsw_r__ -> tursthen..tonbe.tnswhre. with log-joint of -97.29483466250414
i__at_._a_h_n_._e_r_i_g -> in.ath.tanhen..te.reing with log-joint of -75.00721203404348
q___t.___z._____t.__.___.__. -> quret.thez.the.tt.th.the.th. with log-joint of -98.11639027301912


#### 4 - How to improve the model

The model is assuming that a letter only depends on the previous letter, which makes the model skip the other letters to infer a letter. This dependence can be increased to take more letters into account which makes the inference process skip less letter and capture the language model better.